In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/sfo30.tif
/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/innsbruck7.tif
/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/innsbruck20.tif
/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/bloomington30.tif
/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/bellingham11.tif
/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/bloomington21.tif
/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/innsbruck32.tif
/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/innsbruck10.tif
/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/innsbruck6.tif
/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/bloomington23.tif
/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/sfo18.tif
/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/sfo23.tif
/kaggle/input/aerialimagedataset/AerialImageDataset/t

In [5]:
!mkdir /kaggle/working/aerialimage

In [6]:
# create a folders to move train and validation files

!mkdir /kaggle/working/aerialimage/train_input
!mkdir /kaggle/working/aerialimage/train_target

!mkdir /kaggle/working/aerialimage/val_input
!mkdir /kaggle/working/aerialimage/val_target

In [7]:
# create a folders to move train and validation files

!mkdir /kaggle/working/aerialimage/train_input/input_data
!mkdir /kaggle/working/aerialimage/train_target/target_data

!mkdir /kaggle/working/aerialimage/val_input/input_data
!mkdir /kaggle/working/aerialimage/val_target/target_data

In [8]:
import os
import glob

# from osgeo import gdal
import rasterio
import shutil

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random

from PIL import Image

In [9]:
# Check if TensorFlow is using GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        print(f"GPU Device Name: {device.name}")
else:
    print("No GPU devices found. Make sure TensorFlow GPU is properly installed.")

# Check TensorFlow version
print("TensorFlow Version:", tf.__version__)

GPU Device Name: /physical_device:GPU:0
GPU Device Name: /physical_device:GPU:1
TensorFlow Version: 2.11.0


In [10]:
os.environ["PYTHONHASHSEED"] = str(23)
np.random.seed(23)
tf.random.set_seed(23)

In [11]:
batch_size = 5
lr = 1e-4
epochs = 50
height =  512
width = 512
buffer_size = 1000

In [12]:
# move input images to the folder

org_train_input = '/kaggle/input/aerialimagedataset/AerialImageDataset/train/images/'
new_train_input = '/kaggle/working/aerialimage/train_input/input_data'
new_val_input = '/kaggle/working/aerialimage/val_input/input_data'

input_img_paths = []

for dirname, _, filenames in os.walk(org_train_input):
    for filename in filenames:
        input_img_paths.append(os.path.join(dirname, filename))

input_img_paths = sorted(input_img_paths)

random.Random(23).shuffle(input_img_paths)
    
input_img_paths[0:2]


['/kaggle/input/aerialimagedataset/AerialImageDataset/train/images/chicago5.tif',
 '/kaggle/input/aerialimagedataset/AerialImageDataset/train/images/kitsap13.tif']

In [13]:
# copy 40 input images to validation folder

val_num = 40 ## 20% of train dataset 

for file in input_img_paths[:val_num]:
    shutil.copy(file, new_val_input)
    
for file in input_img_paths[val_num:]:
    shutil.copy(file, new_train_input)

In [14]:
# move target images to the folder

org_train_target = '/kaggle/input/aerialimagedataset/AerialImageDataset/train/gt/'
new_train_target = '/kaggle/working/aerialimage/train_target/target_data'
new_val_target = '/kaggle/working/aerialimage/val_target/target_data'

target_img_paths = []

for dirname, _, filenames in os.walk(org_train_target):
    for filename in filenames:
        target_img_paths.append(os.path.join(dirname, filename))

target_img_paths = sorted(target_img_paths)

random.Random(23).shuffle(target_img_paths)

target_img_paths[0:2]

['/kaggle/input/aerialimagedataset/AerialImageDataset/train/gt/chicago5.tif',
 '/kaggle/input/aerialimagedataset/AerialImageDataset/train/gt/kitsap13.tif']

In [15]:
# copy 40 target images to validation folder

for file in target_img_paths[:val_num]:
    shutil.copy(file, new_val_target)
    
for file in target_img_paths[val_num:]:
    shutil.copy(file, new_train_target)

In [16]:
# check whether folder names in input and target folders match

new_train_target = '/kaggle/working/aerialimage/train_target/target_data'
new_val_target = '/kaggle/working/aerialimage/val_target/target_data'

new_train_input = '/kaggle/working/aerialimage/train_input/input_data'
new_val_input = '/kaggle/working/aerialimage/val_input/input_data'

i = 0

for dirname, _, filenames in os.walk(new_train_input):
    for filename in filenames:
        print(filename)
        i += 1
        if i == 5:
            break
        

k = 0

for dirname, _, filenames in os.walk(new_train_target):
    for filename in filenames:
        print(filename)
        k += 1
        if k == 5:
            break

kitsap33.tif
vienna6.tif
vienna34.tif
austin36.tif
tyrol-w29.tif
kitsap33.tif
vienna6.tif
vienna34.tif
austin36.tif
tyrol-w29.tif


In [17]:
# create an image generator / data augmentation pipeline 
# augment data in train dataset only

BATCH_SIZE = 5
TARGET_SIZE = (512,512)
seed = 23
patch_size = (500, 500)  # Specify the desired patch size
overlap = 250  # Specify the desired overlap size, 50% for this case

train_datagen = ImageDataGenerator(rescale=1./255,
                                   samplewise_std_normalization=False, 
                                   horizontal_flip = True, 
                                   vertical_flip = False, # 1st run : False, 2nd run: True
                                   height_shift_range = 0.1, 
                                   width_shift_range = 0.1, 
                                   rotation_range = 3, # 1st run : 3, 2nd run:10
                                   shear_range = 0.01, # 1st run : 0.01, 2nd run: 0.1
                                   fill_mode = 'nearest',
                                   zoom_range = 0.05, #1st run 0.05, 2nd run: 0.1
                                   zca_whitening = True,
                                   zca_epsilon=1e-5, 
#                                    preprocessing_function=create_patches(),
                                   )


target_datagen = ImageDataGenerator(rescale=1./255,
                                   samplewise_std_normalization=False, 
                                   horizontal_flip = True, 
                                   vertical_flip = False, # 1st run : False, 2nd run: True
                                   height_shift_range = 0.1, 
                                   width_shift_range = 0.1, 
                                   rotation_range = 3, # 1st run : 3, 2nd run:10
                                   shear_range = 0.01, # 1st run : 0.01, 2nd run: 0.1
                                   fill_mode = 'nearest',
                                   zoom_range = 0.05, #1st run 0.05, 2nd run: 0.1
                                   zca_whitening = True,
                                   zca_epsilon=1e-5, 
                                   )

val_input_datagen = ImageDataGenerator(rescale=1./255)

val_target_datagen = ImageDataGenerator(rescale=1./255)

/opt/conda/lib/python3.10/site-packages/keras/preprocessing/image.py:1444: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn(


In [18]:
# use data augmentation pipeline and start loading train data from folders

train_generator_input = train_datagen.flow_from_directory(
    '/kaggle/working/aerialimage/train_input',
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    seed = seed

)

train_generator_output = target_datagen.flow_from_directory(
    '/kaggle/working/aerialimage/train_target',
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    color_mode="grayscale",
    seed = seed
)

train_generator = zip(train_generator_input, train_generator_output)

Found 140 images belonging to 1 classes.
Found 140 images belonging to 1 classes.


In [19]:
# use data augmentation pipeline and start loading validation data from folders

val_generator_input = val_input_datagen.flow_from_directory(
    '/kaggle/working/aerialimage/val_input',
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    seed = seed

)

val_generator_output = val_target_datagen.flow_from_directory(
    '/kaggle/working/aerialimage/val_target',
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    color_mode="grayscale",
    seed = seed
)

val_generator = zip(val_generator_input, val_generator_output)

Found 40 images belonging to 1 classes.
Found 40 images belonging to 1 classes.


In [20]:
!mkdir /kaggle/working/aerialimage/model_checkpoint
!mkdir /kaggle/working/aerialimage/model_log

In [21]:
# 1st run
model_file = '/kaggle/working/aerialimage/model_checkpoint/model_checkpoint.h5'
log_file =  '/kaggle/working/aerialimage/model_log/model_log.csv'

In [22]:
# 2nd run
# model_file = '/kaggle/working/aerialimage/model_checkpoint/model_checkpoint_dataaug_editparam.h5'
# log_file =  '/kaggle/working/aerialimage/model_log/model_log_dataau_editparam.csv'

## Unet

In [23]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model


input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [30]:
def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)

    return 1 - numerator / denominator

In [31]:
# configure the optimizer, loss and metrics for training

# Define your two loss functions
loss1 = losses.BinaryCrossentropy()
loss2 = dice_loss

# Define the weights for each loss function
loss_weights = {
    "output1": 0.5,  # Weight for the first loss function
    "output2": 0.5   # Weight for the second loss function
}

# metrics : iou, accuracy
model.compile(optimizer='adam',
              loss=[loss1, loss2],
              loss_weights=loss_weights,
#               loss = dice_loss,
#               loss='binary_crossentropy',
              metrics=[tf.keras.metrics.IoU(num_classes=2, target_class_ids=[0]), 'accuracy', tf.keras.metrics.MeanIoU(num_classes=2)]) # 1 is the target 

In [32]:
callbacks = [ModelCheckpoint(model_file, verbose=1, save_best_only = True),
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
             CSVLogger(log_file),
             EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
             ]

In [ ]:
# 1st run
history = model.fit(train_generator, 
                    epochs=50, 
                    callbacks=callbacks,
                    validation_data=val_generator,
                    steps_per_epoch = 2*140/BATCH_SIZE,
                    validation_steps = 40/BATCH_SIZE)

Epoch 1/50
56/56 [==============================] - ETA: 0s - loss: 0.7056 - io_u_1: 0.8904 - accuracy: 0.7560 - mean_io_u_1: 0.4478
Epoch 1: val_loss improved from inf to 0.70699, saving model to /kaggle/working/aerialimage/model_checkpoint/model_checkpoint.h5
56/56 [==============================] - 195s 3s/step - loss: 0.7056 - io_u_1: 0.8904 - accuracy: 0.7560 - mean_io_u_1: 0.4478 - val_loss: 0.7070 - val_io_u_1: 0.8415 - val_accuracy: 0.9002 - val_mean_io_u_1: 0.4208 - lr: 0.0010
Epoch 2/50
56/56 [==============================] - ETA: 0s - loss: 0.7026 - io_u_1: 0.8899 - accuracy: 0.7612 - mean_io_u_1: 0.4455
Epoch 2: val_loss improved from 0.70699 to 0.70593, saving model to /kaggle/working/aerialimage/model_checkpoint/model_checkpoint.h5
56/56 [==============================] - 178s 3s/step - loss: 0.7026 - io_u_1: 0.8899 - accuracy: 0.7612 - mean_io_u_1: 0.4455 - val_loss: 0.7059 - val_io_u_1: 0.8280 - val_accuracy: 0.8901 - val_mean_io_u_1: 0.4140 - lr: 0.0010
Epoch 3/50
56/

In [3]:
model_loaded = tf.keras.models.load_model(model_file)
# model_loaded.summary()

NameError: name 'model_file' is not defined

In [ ]:
def plot_metrics(metric_name, title, ylim=5):
#   '''plots a given metric from the model history'''
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(model_history.history[metric_name],color='blue',label=metric_name)
    plt.plot(model_history.history['val_' + metric_name],color='green',label='val_' + metric_name)

In [ ]:
# Plot the training and validation loss
plot_metrics("loss", title="Training vs Validation Loss", ylim=1)

## Model Prediction

In [ ]:
# move input images to the folder

org_test_input = '/kaggle/input/aerialimagedataset/AerialImageDataset/test/images/'
new_test_input = '/kaggle/working/aerialimage/train_input/input_data'


input_img_paths = []

for dirname, _, filenames in os.walk(org_test_input):
    for filename in filenames:
        input_img_paths.append(os.path.join(dirname, filename))

input_img_paths = sorted(input_img_paths)

random.Random(23).shuffle(input_img_paths)
    
input_img_paths[0:2]

In [ ]:
# copy 40 input images to validation folder

for file in input_img_paths:
    shutil.copy(file, new_test_input)


In [ ]:
test_input_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:

# Prediction Utilities

def create_mask(pred_mask):
#   '''
#   Creates the segmentation mask by getting the channel with the highest probability. Remember that we
#   have 3 channels in the output of the UNet. For each pixel, the predicition will be the channel with the
#   highest probability.
#   '''
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0].numpy()

def make_predictions(image, num=1):
#   '''
#   Feeds an image to a model and returns the predicted mask.
#   '''

    pred_mask = model.predict(image)
    pred_mask = create_mask(pred_mask)

    return pred_mask

In [ ]:
def display(display_list,titles=[], display_string=None):
#   '''displays a list of images/masks'''

  plt.figure(figsize=(15, 15))

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(titles[i])
        plt.xticks([])
        plt.yticks([])
        if display_string and i == 1:
              plt.xlabel(display_string, fontsize=12)
        img_arr = tf.keras.preprocessing.image.array_to_img(display_list[i])
        plt.imshow(img_arr)

    plt.show()


In [ ]:
def show_predictions(dataset=None, num=1):
    if dataset:
        for imagein dataset.take(num):
            pred_mask = model.predict(image)
            display([image[0], create_mask(pred_mask)])